<a href="https://colab.research.google.com/github/2series/amortTable/blob/main/AmortTableComputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import numpy as np
import numpy_financial as npf
import pandas as pd
from IPython.display import display, HTML

In [ ]:
# Input validation rules for principal, annual rate, loan term, and number of payments per year
class InputValidation:
    """
    This class validates the inputs (Principal, annual rate, term in years and number of payments per year)
    Principal : Validates if input Loan amount is a positive integer
    Annual Rate : Validates if the Annual rate is > 0
    Term in Years (per) : Validates if term is a positive finite integer
    No. of Payments (nper) : Validates if nper is a positive finite integer
    """
    
    def __init__(self, inputvar):
        self.inputvar = inputvar
    
    # Method for checking if the input variable is a positive integer value (Eg. Principal)
    def check_positive_integer(self):
        while True:
            try:
                tempvar = float(input(f"\nPlease Enter a positive integer value for {self.inputvar}:"))
                if tempvar.is_integer() and tempvar > 0:
                    break 
                else:
                    print(f"\n{self.inputvar} should be a positive integer value!")
            except:
                print(f"\n{self.inputvar} is not a number!")
    
        return tempvar

    # Method for checking if the input variable is a positive value (Eg. Annual interest rate)
    def check_positive_float(self):
        while True:
            try:
                tempvar = float(input(f"\nPlease Enter a positive integer value for {self.inputvar}:"))
                if tempvar > 0 and tempvar < 100:
                    break
                else:
                    print(f"\n{self.inputvar} should be a positive value greater than 0 and less than 100!")
            except:
                print("\nNot a valid Option!! Try again..")
        return tempvar
        
    # Method for checking if the input variable is a finite term (Eg.Loan term and No. of payments per year)
    def finiteterm_check(self):
        while True:
            try:
                tempvar = float(input(f"\nPlease Enter a positive integer value for {self.inputvar}:"))
                if tempvar.is_integer() and tempvar > 0 and tempvar < 100:
                    break 
                else:
                    print(f"\n{self.inputvar} should be a positive integer value less than 100!")
            except:
                print(f"\n{self.inputvar} is not a number!")
    
        return tempvar

In [ ]:
# This function produces a summary table for the loan
def generate_summary(principal, initial_rate, per, nper):
    mysum = generate_payment_schedule(principal, initial_rate, per, nper)
    total_interest = mysum["Interest Expense"].sum().round(2)
    pmt = -npf.pmt(initial_rate / nper, per * nper, principal)
    total = total_interest + principal
    
    mymatrix = pd.DataFrame(['R' + str(pmt.round(2)),'R'+ str(total_interest.round(2)),'R'+ str(total.round(2))],
                            columns = [""],
                            index = ["Payment per period", 
                                     "Total Interest", 
                                     "Total Payments"])
    
    return mymatrix

In [ ]:
# This function produces a matrix at various rates and terms
def generate_matrix(principal, paymentfreq, initial_rate, ratestep, period, periodstep):
    
    # Preview a matrix of 6 x 9 matrix with payments at various rates and terms 
    # Declaring Row iterator
    termrows = [i for i in range(5, 45, periodstep)]

    # Declaring Column Iterator
    ratecolumn = [(round(j * 100, 3)) for j in np.arange(initial_rate - 2/100, initial_rate, ratestep)]  +\
    [(round(j * 100, 3)) for j in np.arange(initial_rate + ratestep, initial_rate + 2.25/100, ratestep)] 
    
    # Naming axis and index
    mymatrix = pd.DataFrame(columns= ratecolumn, index=termrows).rename_axis("Annual Rate", axis=1)    
    mymatrix["Period"] = termrows
    mymatrix.set_index("Period", inplace=True)
    
    # Preview Payment matrix for list of period and rates
    for i in termrows:
        for j in ratecolumn:
            mymatrix.at[i, j] = -npf.pmt(float(j / 100) / paymentfreq, i * paymentfreq, principal)
            
    return (mymatrix)

In [ ]:
# This function produces a amortization schedule
def generate_payment_schedule(principal, annual_interest_rate, per, nper):

    # Declaration of variables
    periodic_interest_rate = annual_interest_rate / nper
    no_of_payments = nper * per
    
    # Defining the structure of table
    columnnames = ["Period", 
                   "Opening Balance", 
                   "Payment", 
                   "Interest Expense", 
                   "Repayment of Principal", 
                   "Closing Balance"]

    # Complete static columns & index
    period = [i for i in range(1, no_of_payments + 1)]
 
    # Initialization of the DataFrame
    mymatrix = pd.DataFrame(columns=columnnames, index=period)
    
    # Compute the values for the table 
    mymatrix.at[1, "Opening Balance"] = principal
    mymatrix["Period"] = period
    mymatrix.set_index("Period", inplace=True)
    mymatrix["Payment"] = -npf.pmt(periodic_interest_rate, no_of_payments, principal)
    mymatrix["Interest Expense"] = -npf.ipmt(periodic_interest_rate, mymatrix.index, no_of_payments, principal)
    mymatrix["Repayment of Principal"] = -npf.ppmt(periodic_interest_rate, mymatrix.index, no_of_payments, principal)
    
    # Compute dynamic Amortization Schedule
    for i in period:
        if i > 1:
            mymatrix["Closing Balance"] = mymatrix["Opening Balance"] - mymatrix["Repayment of Principal"]
            mymatrix.at[i, "Opening Balance"] = mymatrix.at[i - 1, "Closing Balance"]
        if mymatrix.at[i, "Opening Balance"] - mymatrix.at[i, "Repayment of Principal"] < 0.1:
            mymatrix.at[i, "Closing Balance"] = 0

    mymatrix.at[1, "Opening Balance"] = principal
    return (mymatrix)

In [ ]:
def amortization_table():
    """ Calculate the loan amortization schedule given the loan details

     Arguments:
        principal: amount borrowed
        annual_interest_rate: annual interest rate for the loan
        per: number of years for the loan
        nper: number of payments in a year

    Returns:
        matrix : returns a 6 x 9 matrix of payments at various rates and term
        schedule: amortization schedule as a pandas dataframe
        summary: pandas dataframe that summarizes the payoff information
    """
    
    # Take User Input     
    principalval = InputValidation("Principal")
    principal = principalval.check_positive_integer()

    rateval = InputValidation("Annual Interest Rate")
    annual_interest_rate = rateval.check_positive_float() / 100
    
    pervalidation = InputValidation("Loan Term in years")
    per = int(pervalidation.finiteterm_check())

    npervalidation = InputValidation("No of Payments per year")
    nper = int(npervalidation.finiteterm_check())
    
    # Input information on the loan
    loan_df = pd.DataFrame(["R" + str(principal), str(annual_interest_rate * 100) + '%', per, nper],
                            columns=[""],
                            index=["Loan Amount", 
                                   "Annual Rate of Interest", 
                                   "Number of Years", 
                                   "Payments per Year"])
    

    # Payment at various rates vs term
    matrix = generate_matrix(principal, nper, annual_interest_rate, 0.005, per, 5)

    # Amortization Schedule
    schedule = generate_payment_schedule(principal, annual_interest_rate, per, nper)
    summary = generate_summary(principal, annual_interest_rate, per, nper)
    
    display(pd.concat([loan_df, summary], axis=0).style.set_caption("Loan Summary").set_table_styles([
                                      {'selector' : '',
                                       'props' : [('background-color', 'white'),
                                                  ('border', '2px solid black')]},
                                      {'selector': 'caption',
                                       'props': [('color', '#4f4646'),
                                                 ('font-size', '16px'),
                                                 ('text-align', 'center')]}]))
    
    print("\n \033[1m Based on the information you entered, your payment is {} for {} years with a rate of {}%\033[1m".\
          format(summary.at["Payment per period",""], per, annual_interest_rate * 100))
    
    display(schedule.style.set_caption("Payment Schedule").set_table_styles([
                            {'selector' : '',
                            'props' : [('background-color', 'white'),
                                       ('border', '2px solid black')]},
                            {'selector': 'caption',
                            'props': [('color', '#4f4646'),
                                      ('font-size', '16px'),
                                      ('text-align', 'center')]}]).format('R{:,.2f}'))

In [ ]:
amortization_table()

,
Loan Amount,R100.0
Annual Rate of Interest,10.0%
Number of Years,1
Payments per Year,12
Payment per period,R8.79
Total Interest,R5.5
Total Payments,R105.5



  Based on the information you entered, your payment is R8.79 for 1 years with a rate of 10.0%


,Opening Balance,Payment,Interest Expense,Repayment of Principal,Closing Balance
Period,,,,,
1,R100.00,R8.79,R0.83,R7.96,R92.04
2,R92.04,R8.79,R0.77,R8.02,R84.02
3,R84.02,R8.79,R0.70,R8.09,R75.93
4,R75.93,R8.79,R0.63,R8.16,R67.77
5,R67.77,R8.79,R0.56,R8.23,R59.54
6,R59.54,R8.79,R0.50,R8.30,R51.24
7,R51.24,R8.79,R0.43,R8.36,R42.88
8,R42.88,R8.79,R0.36,R8.43,R34.45
9,R34.45,R8.79,R0.29,R8.50,R25.94
